In [1]:
import numpy as np
import os
from utils.tools import *
import keras
import matplotlib.pyplot as plt
%matplotlib inline
from multiprocessing import Process,Queue,Value
from generator import DataGenerator
from utils.tools import sample 

Using TensorFlow backend.


In [2]:
import gym
from collections import deque
env=gym.make('CartPole-v0')
env.reset()
q=deque()
for i in range(4):
    q.append(reshape(env.step(1)[0]))
np.stack(q, axis=-1).shape

(4, 1, 4)

In [3]:
config=load_config('config.yaml')
print (config)

{'actions': 2, 'image': 'not_img', 'bottom_right_x': 'None', 'bottom_right_y': 'None', 'top_left_x': 'None', 'top_left_y': 'None', 'decay': 1e-06, 'env_name': 'CartPole-v0', 'eps': 0.1, 'exploration': 'greedy', 'gain': 5, 'gamma': 0.9, 'gpu': False, 'input_shape': (4, 1, 4), 'max_episode_steps': 1000, 'nstack': 4, 'path': 'Policy/', 'start_from_pretrained': 0, 'learning_rate': 0.001, 'Total_Q_Iterations': 50, 'sample_size': 8000, 'target_update_iterations': 5, 'regess_for_epochs': 40}


In [4]:
replay_buffer=Queue(maxsize=10000)
saved_new=Value('i',config['start_from_pretrained'])
shared_objs={'replay_buffer':replay_buffer,'load_network':saved_new}
G=DataGenerator(config)

In [5]:
P=Process(target=G.generate_data,args=(shared_objs,))
P.start()

In [15]:
replay_buffer.qsize()

0

In [6]:
import tensorflow as tf
from Policy.PolicyNetwork import DQN
if config['gpu']==True:
            from keras import backend as K
            config1 = tf.ConfigProto()
            config1.gpu_options.allow_growth=True
            sess = tf.Session(config=config1)
            K.set_session(sess)
model = DQN(input_shape=config['input_shape'],num_outputs=config['actions'],
          lr=config['learning_rate'],
          arch_config_file=os.path.join(config['path'],"qnetwork.csv"))
model.construct_model(training=True)
if (config['start_from_pretrained']):
    model.Model.load_weights('weights.h5')

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 4, 1, 4)      0                                            
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 4, 1, 4)      16          input_1[0][0]                    
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 4, 1, 32)     160         batch_normalization_1[0][0]      
__________________________________________________________________________________________________
batch_normalization_2 (BatchNor (None, 4, 1, 32)     128         conv2d_1[0][0]                   
__________________________________________________________________________________________________
activation

C:\Users\MuhammadUsman\PycharmProjects\Deep-Q-Learning\Policy\PolicyNetwork.py:36: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=[<tf.Tenso..., outputs=Tensor("la...)`
  self.Model = keras_model(inputs=[state_inputs,action_one_hot_inputs], output=pred_q_a)


In [8]:
for i in range(config['Total_Q_Iterations']):
    print ('Q_Iteration: {}'.format(i+1))
    for j in range(config['target_update_iterations']):
        print ('target_update_iterations {}'.format(i))
        X,y,a=sample(replay_buffer,config['sample_size'])
        print (X.shape)
        model.Model.fit([X,keras.utils.to_categorical(a,num_classes=2)],y,epochs=config['regess_for_epochs'],batch_size=512,
                        verbose=2)
    model.Model.save_weights('weights.h5')
    with saved_new.get_lock():
        saved_new.value = 1

Q_Iteration: 1
target_update_iterations 0


(8000, 4, 1, 4)


Epoch 1/40


 - 2s - loss: 0.5356


Epoch 2/40


...

 - 1s - loss: 7.3723e-04


Epoch 11/40


 - 1s - loss: 6.4758e-04


Epoch 12/40


KeyboardInterrupt: 